In [4]:
import pandas as pd

In [5]:
import pydot
import time

In [6]:
from pprint import pprint

In [7]:
import numpy as np

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss
from scipy.special import kl_div
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In this notebook we will be testing code to construct the random forest using python libraries

The first step will be reading in our data

In [10]:
Motif_present = pd.read_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Motif_Present.tsv',sep="\t")

In [11]:
Motif_present.head(5)

,gene_id,ec_id,ko_id,map_id,species,MEME-1,MEME-4,MEME-5,MEME-10,MEME-7,...,MEME-42,MEME-87,MEME-62,MEME-81,MEME-80,MEME-79,MEME-46,MEME-94,MEME-61,MEME-84
0,bis:DXK01_000010,ec:5.4.99.2,ko:K01847,"['path:map00280', 'path:map00630', 'path:map00...",bis,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,bis:DXK01_000015,ec:5.4.99.2,ko:K01847,"['path:map00280', 'path:map00630', 'path:map00...",bis,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,bis:DXK01_000035,ec:6.1.1.6,ko:K04567,[path:map00970],bis,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,bis:DXK01_000040,ec:1.1.1.94,ko:K00057,"['path:map00564', 'path:map01110']",bis,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,bis:DXK01_000045,ec:5.3.1.9,ko:K01810,"['path:map00010', 'path:map00030', 'path:map00...",bis,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [12]:
Motif_number = pd.read_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Motif_Number.tsv',sep="\t")

In [13]:
Motif_number.head(5)

,gene_id,ec_id,ko_id,map_id,species,MEME-1,MEME-4,MEME-5,MEME-10,MEME-7,...,MEME-42,MEME-87,MEME-62,MEME-81,MEME-80,MEME-79,MEME-46,MEME-94,MEME-61,MEME-84
0,eco:b3994,ec:4.1.99.17,ko:K03147,"['path:map00730', 'path:map01100']",eco,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,bis:DXK01_001240,ec:4.1.99.17,ko:K03147,"['path:map00730', 'path:map01100']",bis,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,bis:DXK01_013610,ec:1.7.99.1,ko:K05601,"['path:map00910', 'path:map01100']",bis,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,bis:DXK01_015090,ec:5.4.2.11,ko:K01834,"['path:map00010', 'path:map00260', 'path:map00...",bis,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,eco:b0755,ec:5.4.2.11,ko:K01834,"['path:map00010', 'path:map00260', 'path:map00...",eco,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
Nodes_present = pd.read_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Nodes_Present.tsv',sep="\t")

In [15]:
Nodes_present.head(5)

,gene_id,ec_id,ko_id,map_id,species,n1,n2,n3,n4,n5,...,n3151,n3152,n3153,n3154,n3155,n3156,n3157,n3158,n3159,n3160
0,bis:DXK01_000010,ec:5.4.99.2,ko:K01847,"['path:map00280', 'path:map00630', 'path:map00...",bis,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,bis:DXK01_000015,ec:5.4.99.2,ko:K01847,"['path:map00280', 'path:map00630', 'path:map00...",bis,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,bis:DXK01_000035,ec:6.1.1.6,ko:K04567,[path:map00970],bis,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,bis:DXK01_000040,ec:1.1.1.94,ko:K00057,"['path:map00564', 'path:map01110']",bis,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,bis:DXK01_000045,ec:5.3.1.9,ko:K01810,"['path:map00010', 'path:map00030', 'path:map00...",bis,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [16]:
Nodes_number = pd.read_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Nodes_Number.tsv',sep="\t")

In [17]:
Nodes_number.head(5)

,gene_id,ec_id,ko_id,map_id,species,n1,n2,n3,n4,n5,...,n3151,n3152,n3153,n3154,n3155,n3156,n3157,n3158,n3159,n3160
0,bis:DXK01_000010,ec:5.4.99.2,ko:K01847,"['path:map00280', 'path:map00630', 'path:map00...",bis,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,bis:DXK01_000015,ec:5.4.99.2,ko:K01847,"['path:map00280', 'path:map00630', 'path:map00...",bis,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bis:DXK01_000035,ec:6.1.1.6,ko:K04567,[path:map00970],bis,0,0,13,0,0,...,0,0,0,0,0,0,0,0,0,0
3,bis:DXK01_000040,ec:1.1.1.94,ko:K00057,"['path:map00564', 'path:map01110']",bis,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
4,bis:DXK01_000045,ec:5.3.1.9,ko:K01810,"['path:map00010', 'path:map00030', 'path:map00...",bis,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0


As we can see we have 4 different training data sets that either show gene information of Motifs or Nodes and the Presence or Number of motifs in each. We will be testing these four data sets against eachother to reveal which is the most accurate predictor of Pathway.

In [18]:
Tree_present = pd.read_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Tree_Present.tsv',sep="\t")
Tree_present.head(5)

,gene_id,ec_id,ko_id,map_id,species,n1,n2,n3,n4,n5,...,n3151,n3152,n3153,n3154,n3155,n3156,n3157,n3158,n3159,n3160
0,bis:DXK01_000010,ec:5.4.99.2,ko:K01847,"['path:map00280', 'path:map00630', 'path:map00...",bis,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,bis:DXK01_000015,ec:5.4.99.2,ko:K01847,"['path:map00280', 'path:map00630', 'path:map00...",bis,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,bis:DXK01_000035,ec:6.1.1.6,ko:K04567,[path:map00970],bis,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,bis:DXK01_000040,ec:1.1.1.94,ko:K00057,"['path:map00564', 'path:map01110']",bis,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,bis:DXK01_000045,ec:5.3.1.9,ko:K01810,"['path:map00010', 'path:map00030', 'path:map00...",bis,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [19]:
print('Shape of the data \nMotif Present:',Motif_present.shape,'\nMotif Numbers:',Motif_number.shape,'\nNodes Present:',Nodes_present.shape,'\nNodes Numbers:', Nodes_number.shape )

Shape of the data 
Motif Present: (1523, 102) 
Motif Numbers: (1523, 102) 
Nodes Present: (1581, 3165) 
Nodes Numbers: (1581, 3165)


It seems that there are about 50 missing genes in Motif dataset, this is likely because no motifs could be identified in those genes but they still pathway annotations. Regardless we will keep that data in.

In [20]:
Nodes_number.describe()

,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,...,n3151,n3152,n3153,n3154,n3155,n3156,n3157,n3158,n3159,n3160
count,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.0,...,1581.00000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000,1581.000000
mean,0.003795,0.003163,0.008223,0.003163,0.002530,0.000633,0.001898,0.005060,0.003163,0.0,...,0.00506,0.006325,0.006325,0.002530,0.006325,0.001265,0.003795,0.003795,0.001265,0.006325
std,0.150899,0.125749,0.326947,0.125749,0.100599,0.025150,0.075449,0.201198,0.125749,0.0,...,0.17782,0.177779,0.181305,0.079515,0.181305,0.035556,0.128224,0.106668,0.050300,0.177779
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,6.000000,5.000000,13.000000,5.000000,4.000000,1.000000,3.000000,8.000000,5.000000,0.0,...,7.00000,5.000000,6.000000,3.000000,6.000000,1.000000,5.000000,3.000000,2.000000,5.000000


In [21]:
Motif_present.dtypes

gene_id    object
ec_id      object
ko_id      object
map_id     object
species    object
            ...  
MEME-79      bool
MEME-46      bool
MEME-94      bool
MEME-61      bool
MEME-84      bool
Length: 102, dtype: object

Looks like there is not too much going on with the nodes, they are pretty random, no patterns, and the present dfs are bools so no need to one hot encode.

Now let us do several things, first convert each of the training sets into numpy arrays to work with the thing. Then we want to separate the features and targets.

In [22]:
Motif_present_targetEC = np.array(Motif_present['ec_id']) #Lets start with predicting the EC and then we can try mapID later
Motif_number_targetEC = np.array(Motif_number['ec_id'])
Nodes_present_targetEC = np.array(Nodes_present['ec_id'])
Nodes_number_targetEC = np.array(Nodes_number['ec_id'])
Tree_present_targetEC = np.array(Tree_present['ec_id'])
Motif_number_targetEC.size

1523

In [23]:
Motif_present_targetMap = np.array(Motif_present['map_id'])
Motif_number_targetMap = np.array(Motif_number['map_id'])
Nodes_present_targetMap = np.array(Nodes_present['map_id'])
Nodes_number_targetMap = np.array(Nodes_number['map_id'])
Tree_present_targetMap = np.array(Tree_present['map_id'])
Motif_present_targetMap.size

1523

In [24]:
colDrop = ['gene_id','ec_id','ko_id','map_id','species']

In [25]:
Motif_present_features = Motif_present.drop(colDrop,axis = 1)

In [26]:
Motif_present_features.head(5)

,MEME-1,MEME-4,MEME-5,MEME-10,MEME-7,MEME-11,MEME-24,MEME-6,MEME-17,MEME-13,...,MEME-42,MEME-87,MEME-62,MEME-81,MEME-80,MEME-79,MEME-46,MEME-94,MEME-61,MEME-84
0,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
Motif_number_features = Motif_number.drop(colDrop,axis = 1)
Nodes_present_features = Nodes_present.drop(colDrop,axis = 1)
Nodes_number_features = Nodes_number.drop(colDrop,axis = 1)
Tree_present_features = Tree_present.drop(colDrop,axis = 1)

Lets also save the names of the features of each, why not (present and number feature names should be the same)

In [28]:
Motif_present_features_names = list(Motif_present_features.columns)
Nodes_present_features_names = list(Nodes_present_features.columns)

And now lastly we will convert the features into numpy arrays

In [29]:
Motif_present_features_array = np.array(Motif_present_features)
Motif_number_features_array = np.array(Motif_number_features)
Nodes_present_features_array = np.array(Nodes_present_features)
Nodes_number_features_array = np.array(Nodes_number_features)
Tree_present_features_array = np.array(Tree_present_features)

    Now we can start coding the random forest algorithm. Since we are just doing this to get a base on the random forest design, we will only use one dataframe 'Nodes_present' to code it and then we will do all with the cross validation function

In [30]:
train_features, test_features, train_labels, test_labels = train_test_split(Motif_number_features_array,Motif_number_targetEC,test_size = 0.25, random_state = 42)

In [29]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1142, 97)
Training Labels Shape: (1142,)
Testing Features Shape: (381, 97)
Testing Labels Shape: (381,)


The tutorial mentions a baseline but we are predicting non-numeric so loss and accuracy is going to be the most important for us. Now we can import the regressor model and fit to our training data.

In [30]:
#rf = RandomForestRegressor(n_estimators = 1000, random_state = 42) #think I need the classifier instead
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [ ]:
rf.fit(train_features,train_labels)

It looks like we need to have removed the gene id column, which makes sense because otherwise it would think that gene ID is a predictor

In [ ]:
predictions = rf.predict(test_features)
len(predictions)

In [ ]:
#caclulate the absolute errors 
# errors = abs(predictions - test_labels)
errors = 0
for x in range(0,len(predictions)):
    if predictions[x] == test_labels[x]:
        errors = errors + 1

In [ ]:
print('accuracy was ',(errors/len(test_labels))*100,'%')

Obviously that is very low but we aren't using any changing of parameters or anything and we are only using one model. 
For fun though, let's visualize one of the trees.

In [ ]:
# tree = rf.estimators_[5]
# export_graphviz(tree, out_file='/home/robbenm/LuberLab/crispri/EcoBisTest/testtree.dot',feature_names=Motif_present_features_names,rounded = True, precision = 1)
# (graph,) = pydot.graph_from_dot_file('/home/robbenm/LuberLab/crispri/EcoBisTest/testtree.dot')
# graph.write_png('/home/robbenm/LuberLab/crispri/EcoBisTest/testtree.png')

nvm, the plotting does not work on here but I think I get it, let's move on and predict feature importance

In [ ]:
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(Motif_present_features_names, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Good, at the very least no motif is more important than another in determining pathway

Let's move on to the hyperparameter tuning with crossfold cv for this Eco-Bis set. Essentially the goal behind this is to have the algorithm scan through the different parameters while using different training/test sets to show us what parameters and training sets will provide the best results.

In [36]:
#First make a new Random Forest Classifier
rf = RandomForestClassifier(random_state = 42)

In [37]:
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


These are all of the parameters used by the random classifier to classify. We want to come up with a bunch of variables for each and put them in a grid so that the cv can iterate through them. The parameters we will use are:




In [38]:
n_estimators =  [100, 500, 1000, 2500, 5000, 7500, 10000] # num of trees

In [39]:
max_features = ['auto','sqrt'] # Number of features considered at each split

In [40]:
max_depth = [int(x) for x in np.linspace(10,200,num = 10)] #max levels in each tree
max_depth.append(None)

In [41]:
min_samples_split = [2,5,10] # Min num samples to split a node

In [42]:
min_samples_leaf = [1, 2, 4] # Min num samples required at each leaf node

In [43]:
bootstrap = [True, False]

In [46]:
random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 200, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 500, 1000, 2500, 5000, 7500, 10000]}


We will first do a random search that will randomly apply settings, because it is too computationally costly to do up to 4,320 runs of the model fitting, we would rather narrow it down first. we will continue to use just the Model_present at first but will need to create a function to easily plug in all of them.


In [47]:
rf = RandomForestClassifier()
#Perform random search using 5 fold cv, accross 100 different settings using all available cores
Motif_present_rf = RandomizedSearchCV(estimator=rf, 
                               param_distributions=random_grid,
                               n_iter=100, 
                               cv = 5, 
                               verbose =2, 
                               random_state=42,
                               n_jobs = 32)
# Motif_present_rf.fit(train_features, train_labels)

It appears that when performing this multicored, each core can use up to 30 GB of memory, meaning for 16 threads you need at least 500 GB active. At 500 folds with 16 GB it took 30 min so doubling it would be adviseable for the 12+ runs I will have to do and larger data set. 
Any parallel error is due to lack of memory.

In [48]:
# Motif_present_rf.best_params_

In [49]:
# predictions = Motif_present_rf.best_estimator_.predict(test_features)
# len(predictions)
# errors = 0
# for x in range(0,len(predictions)):
#     if predictions[x] == test_labels[x]:
#         errors = errors + 1
        
# print('accuracy was ',(errors/len(test_labels))*100,'%')

Oof, we only increased accuracy by 1%, yikes

In [62]:
#Now we create a function that we can just input our table and it will completely make the model and test it.
#we will not use the randomCV since right now we are just comparing but we will take the best options from
#the previous randomCV and apply them to this
def make_model(features,labels,ncores = 1):
    train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                                labels,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
    #Make the random forest
    rf = RandomForestClassifier(random_state = 42,
                                n_estimators = 5000,
                                min_samples_split = 2,
                                min_samples_leaf = 1,
                                max_features = 'sqrt',
                                max_depth = 10,
                                bootstrap = True,
                               oob_score = True)
#     cv_score = cross_val_score(rf, features, labels,cv = 5,n_jobs = 16)
    rf.fit(train_features,train_labels)
    
    
    print('Training score: ', rf.score(train_features, train_labels))
    print('OOB score: ', rf.oob_score_)
    print('Test score: ', rf.score(test_features, test_labels))
    
    predictions = rf.predict(test_features)
    y_pred = rf.predict_proba(test_features)
    lloss = log_loss(test_labels,y_pred,labels=rf.classes_)
    conf_mat = confusion_matrix(test_labels, predictions)
    FP = conf_mat.sum(axis=0) - np.diag(conf_mat)  
    FN = conf_mat.sum(axis=1) - np.diag(conf_mat)
    TP = np.diag(conf_mat)
    TN = conf_mat.sum() - (FP + FN + TP)

#     # Sensitivity, hit rate, recall, or true positive rate
#     TPR = TP/(TP+FN)
#     # Specificity or true negative rate
#     TNR = TN/(TN+FP) 
#     # Precision or positive predictive value
#     PPV = TP/(TP+FP)
#     # Negative predictive value
#     NPV = TN/(TN+FN)
#     # Fall out or false positive rate
#     FPR = FP/(FP+TN)
#     # False negative rate
#     FNR = FN/(TP+FN)
#     # False discovery rate
#     FDR = FP/(TP+FP)

#     # Overall accuracy
#     ACC = (TP+TN)/(TP+FP+FN+TN)
    
    print('Average True Pos:',TP.mean())
    print('Average True Neg:',TN.mean())
    print('Average False Pos:',FP.mean())
    print('Average False Neg:',FN.mean())
#     print('True Positive Rate (recall):',TPR.mean())
#     print('True Negative Rate:',TNR.mean())
#     print('Precision:', PPV.mean())
#     print('False Positive Rate:',FPR.mean())
#     print('(meaningless) average computed accuracy:', ACC.mean())
    
    print('Log Loss: ',lloss)
#     print('Cross validation score: ',cv_score.mean())
    return(rf)
    

Now we can use the function to run through all 4 of our training data sets with 1) predicting EC and 2) predicting pathway

In [63]:
Motif_present_rf = make_model(Motif_present_features_array,Motif_present_targetEC)

Training score:  0.7898423817863398
OOB score:  0.0446584938704028
Test score:  0.06824146981627296
Average True Pos: 0.07027027027027027
Average True Neg: 379.0108108108108
Average False Pos: 0.9594594594594594
Average False Neg: 0.9594594594594594
Log Loss:  3.865290615619425


In [64]:
Motif_number_rf = make_model(Motif_number_features_array,Motif_number_targetEC)

Training score:  0.7924693520140105
OOB score:  0.05691768826619965
Test score:  0.047244094488188976
Average True Pos: 0.048
Average True Neg: 379.016
Average False Pos: 0.968
Average False Neg: 0.968
Log Loss:  3.984779055145796


In [65]:
Nodes_present_rf = make_model(Nodes_present_features_array,Nodes_present_targetEC)

Training score:  0.47848101265822784
OOB score:  0.1189873417721519
Test score:  0.13131313131313133
Average True Pos: 0.1452513966480447
Average True Neg: 393.9329608938547
Average False Pos: 0.9608938547486033
Average False Neg: 0.9608938547486033
Log Loss:  3.517048541623167


In [66]:
Nodes_number_rf = make_model(Nodes_number_features_array,Nodes_number_targetEC)

Training score:  0.41434599156118146
OOB score:  0.08945147679324894
Test score:  0.12121212121212122
Average True Pos: 0.13793103448275862
Average True Neg: 393.86206896551727
Average False Pos: 1.0
Average False Neg: 1.0
Log Loss:  3.679297746178915


In [67]:
# Tree_present_rf = make_model(Tree_present_features_array,Tree_present_targetEC) #same results as nodes present

In [68]:
Motif_present_rfmap = make_model(Motif_present_features_array,Motif_present_targetMap)

Training score:  0.7539404553415061
OOB score:  0.05516637478108581
Test score:  0.07349081364829396
Average True Pos: 0.14432989690721648
Average True Neg: 377.21649484536084
Average False Pos: 1.8195876288659794
Average False Neg: 1.8195876288659794
Log Loss:  4.643495694732342


In [69]:
Motif_number_rfmap = make_model(Motif_number_features_array,Motif_number_targetMap)

Training score:  0.7732049036777583
OOB score:  0.053415061295971976
Test score:  0.07349081364829396
Average True Pos: 0.14893617021276595
Average True Neg: 377.09574468085106
Average False Pos: 1.877659574468085
Average False Neg: 1.877659574468085
Log Loss:  4.777378760201549


In [70]:
Nodes_present_rfmap = make_model(Nodes_present_features_array,Nodes_present_targetMap)

Training score:  0.4582278481012658
OOB score:  0.1308016877637131
Test score:  0.13383838383838384
Average True Pos: 0.2535885167464115
Average True Neg: 392.46411483253587
Average False Pos: 1.6411483253588517
Average False Neg: 1.6411483253588517
Log Loss:  4.332446404368004


In [71]:
Nodes_number_rfmap = make_model(Nodes_number_features_array,Nodes_number_targetMap)

Training score:  0.3848101265822785
OOB score:  0.09367088607594937
Test score:  0.11616161616161616
Average True Pos: 0.2169811320754717
Average True Neg: 392.4811320754717
Average False Pos: 1.650943396226415
Average False Neg: 1.650943396226415
Log Loss:  4.427627830776111


In [72]:
# Tree_present_rfmap = make_model(Tree_present_features_array,Tree_present_targetMap)

As it appears, predicting map IDs with nodes resulted in higher accuracy but that is when multiple map ID's are represented together, so if it doesn't predict all map IDs it will result in a loss of accuracy even if it gets one or two. Let's recreate the function above to work with separating and comparing individual map IDs.

In [59]:
#test code for splitting
map1 = Motif_present['map_id'][1]#['path:map00280', 'path:map00630']
map2 = ["'path:map00280'", " 'path:map00630'", " 'path:map00640'",]
map1 = map1.replace('[','').replace(']','').split(',')
print(map1)

["'path:map00280'", " 'path:map00630'", " 'path:map00640'", " 'path:map00720'", " 'path:map01100'", " 'path:map01120'"]


In [60]:
def make_model_count(features,labels,ncores = 1):
    train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                                labels,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
    
    #Make the random forest
    rf = RandomForestClassifier(random_state = 42,
                                n_estimators = 5000,
                                min_samples_split = 2,
                                min_samples_leaf = 1,
                                max_features = 'sqrt',
                                max_depth = 10,
                                bootstrap = True)
    rf.fit(train_features,train_labels)
    predictions = rf.predict(test_features)
    errors = 0
    length = 0
    less = 0
    equal = 0
    more = 0
    for x in range(0,len(predictions)):
        map1 = predictions[x]#['path:map00280', 'path:map00630']
        map2 = test_labels[x]
        map1 = map1.replace('[','').replace(']','').split(',')
        map2 = map2.replace('[','').replace(']','').split(',')
        total = len(set(map1).intersection(map2))
        errors = errors + total
        length = length + len(map2)
        if (total < len(map2)): less = less + 1
        if (total == len(map2)): equal = equal + 1
        if (total < len(map1)): more = more + 1
    print('accuracy was ',errors/length*100,'%')
    print('in ', less/len(test_labels)*100,'% of genes, model did not predict all pathways')
    print('in ', equal/len(test_labels)*100,'% of genes, model predicted all pathways')
    print('in ', more/len(test_labels)*100,'% of genes, model predicted pathways not found in gene')
    return(rf)

In [61]:
Motif_present_rfmapcount = make_model_count(Motif_present_features_array,Motif_present_targetMap)

accuracy was  31.026438569206842 %
in  92.6509186351706 % of genes, model did not predict all pathways
in  7.349081364829396 % of genes, model predicted all pathways
in  91.86351706036746 % of genes, model predicted pathways not found in gene


In [62]:
Motif_number_rfmapcount = make_model_count(Motif_number_features_array,Motif_number_targetMap)

accuracy was  31.642512077294686 %
in  92.6509186351706 % of genes, model did not predict all pathways
in  7.349081364829396 % of genes, model predicted all pathways
in  91.86351706036746 % of genes, model predicted pathways not found in gene


In [63]:
Nodes_present_rfmapcount = make_model_count(Nodes_present_features_array,Nodes_present_targetMap)

accuracy was  18.62673484295106 %
in  86.61616161616162 % of genes, model did not predict all pathways
in  13.383838383838384 % of genes, model predicted all pathways
in  86.61616161616162 % of genes, model predicted pathways not found in gene


In [64]:
Nodes_number_rfmapcount = make_model_count(Nodes_number_features_array,Nodes_number_targetMap)

accuracy was  15.997078159240322 %
in  88.38383838383838 % of genes, model did not predict all pathways
in  11.616161616161616 % of genes, model predicted all pathways
in  88.38383838383838 % of genes, model predicted pathways not found in gene


In [65]:
Importance_data_Motifs = {'Motif_Name': Motif_present_features_names,
                         'Motif_number_rf_EC': Motif_number_rf.feature_importances_,
                         'Motif_number_rf_Map': Motif_number_rfmap.feature_importances_,
                         'Motif_number_rf_Mapcount': Motif_number_rfmapcount.feature_importances_,
                         'Motif_present_rf_EC': Motif_present_rf.feature_importances_,
                         'Motif_present_rf_Map': Motif_present_rfmap.feature_importances_,
                         'Motif_present_rf_Mapcount': Motif_present_rfmapcount.feature_importances_}
Importance_data_Motifs_df = pd.DataFrame(Importance_data_Motifs)
Importance_data_Motifs_df.to_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Motif_importance.csv')
Importance_data_Motifs_df.head(5)

,Motif_Name,Motif_number_rf_EC,Motif_number_rf_Map,Motif_number_rf_Mapcount,Motif_present_rf_EC,Motif_present_rf_Map,Motif_present_rf_Mapcount
0,MEME-1,0.010605,0.010807,0.010807,0.010569,0.011073,0.011073
1,MEME-4,0.008627,0.008157,0.008157,0.009029,0.009014,0.009014
2,MEME-5,0.009501,0.008852,0.008852,0.008883,0.008716,0.008716
3,MEME-10,0.009413,0.009890,0.009890,0.011041,0.010522,0.010522
4,MEME-7,0.009238,0.008730,0.008730,0.008753,0.008887,0.008887


In [66]:
Importance_data_Nodes = {'Nodes_Name': Nodes_present_features_names,
                         'Nodes_number_rf_EC': Nodes_number_rf.feature_importances_,
                         'Nodes_number_rf_Map': Nodes_number_rfmap.feature_importances_,
                         'Nodes_number_rf_Mapcount': Nodes_number_rfmapcount.feature_importances_,
                         'Nodes_present_rf_EC': Nodes_present_rf.feature_importances_,
                         'Nodes_present_rf_Map': Nodes_present_rfmap.feature_importances_,
                         'Nodes_present_rf_Mapcount': Nodes_present_rfmapcount.feature_importances_}
Importance_data_Nodes_df = pd.DataFrame(Importance_data_Nodes)
Importance_data_Nodes_df.to_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Nodes_importance.csv')
Importance_data_Nodes_df.head(5)

,Nodes_Name,Nodes_number_rf_EC,Nodes_number_rf_Map,Nodes_number_rf_Mapcount,Nodes_present_rf_EC,Nodes_present_rf_Map,Nodes_present_rf_Mapcount
0,n1,0.000080,0.000107,0.000107,0.000144,0.000163,0.000163
1,n2,0.000085,0.000094,0.000094,0.000243,0.000224,0.000224
2,n3,0.000312,0.000277,0.000277,0.000253,0.000123,0.000123
3,n4,0.000124,0.000101,0.000101,0.000103,0.000137,0.000137
4,n5,0.000063,0.000068,0.000068,0.000069,0.000074,0.000074


After loss calculations, lets also try giving our models to the adaboost (boosting) models to see if they improve accuracy at all. Let's just give it a wack with the motif numbers model set.

In [67]:
clf = AdaBoostClassifier(n_estimators = 100)
scores = cross_val_score(clf, Motif_number_features_array, Motif_number_targetMap,cv = 5)
scores.mean()

/home/robbenm/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.02888481449525453

Lets run a random search to find favorable hyperparameters. 

In [68]:
pprint(clf.get_params())

{'algorithm': 'SAMME.R',
 'base_estimator': None,
 'learning_rate': 1.0,
 'n_estimators': 100,
 'random_state': None}


We can change the learning rate (0-1), n_estimators (100:10000), and base estimator (decision tree max_depth 1-10). Lets do a random CV to evaluate these parameters on at least the motif_number model.

In [69]:
n_estimators =  [100, 500, 1000, 2500, 5000, 7500, 10000] # num of trees
learning_rate = [0.1,0.2,0.4,0.6,0.8,1,1.25,1.5,2]
base_est = [DecisionTreeClassifier(max_depth=1),DecisionTreeClassifier(max_depth=3),DecisionTreeClassifier(max_depth=6),DecisionTreeClassifier(max_depth=9)]
#Might not be able to run base_estimator max depth in randomfoldcv
random_grid = {'n_estimators': n_estimators,
              'learning_rate': learning_rate}
pprint(random_grid)

{'learning_rate': [0.1, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5, 2],
 'n_estimators': [100, 500, 1000, 2500, 5000, 7500, 10000]}


Now lets run the randomfoldcv

In [70]:
# ada = AdaBoostClassifier()
# Motif_present_ada = RandomizedSearchCV(estimator=ada, 
#                                param_distributions=random_grid,
#                                n_iter=100, 
#                                cv = 5, 
#                                verbose =2, 
#                                random_state=42,
#                                n_jobs = 16)
# Motif_present_ada.fit(train_features, train_labels)

In [1]:
# Motif_present_ada.best_params_

Looks like the best parameters for the adaboosted is using 7500 ntrees and a learning rate of 0.4. We will apply that to a series of models using different max depths. 

In [92]:
dt1 = DecisionTreeClassifier(max_depth=1)
dt3 = DecisionTreeClassifier(max_depth=3)
dt6 = DecisionTreeClassifier(max_depth=6)
dt9 = DecisionTreeClassifier(max_depth=9)
dt12 = DecisionTreeClassifier(max_depth=12)
ada1 = AdaBoostClassifier(base_estimator=dt1,learning_rate=0.4,n_estimators = 7500)
ada3 = AdaBoostClassifier(base_estimator=dt3,learning_rate=0.4,n_estimators = 7500)
ada6 = AdaBoostClassifier(base_estimator=dt6,learning_rate=0.4,n_estimators = 7500)
ada9 = AdaBoostClassifier(base_estimator=dt9,learning_rate=0.4,n_estimators = 7500)
ada12 = AdaBoostClassifier(base_estimator=dt12,learning_rate=0.4,n_estimators = 7500)

In [88]:
# scores = cross_val_score(ada1, Motif_number_features_array, Motif_number_targetEC,cv = 5,n_jobs = 5)
scores.mean()

/home/robbenm/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.00985332182916307

In [89]:
# scores = cross_val_score(ada3, Motif_number_features_array, Motif_number_targetEC,cv = 5,n_jobs = 5)
scores.mean()

/home/robbenm/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.01771786022433132

In [90]:
# scores = cross_val_score(ada6, Motif_number_features_array, Motif_number_targetEC,cv = 5,n_jobs = 5)
scores.mean()

/home/robbenm/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/robbenm/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


0.019687230371009494

In [91]:
# scores = cross_val_score(ada9, Motif_number_features_array, Motif_number_targetEC,cv = 5,n_jobs = 5)
scores.mean()

/home/robbenm/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.02296807592752373

In [93]:
# scores = cross_val_score(ada12, Motif_number_features_array, Motif_number_targetEC,cv = 5,n_jobs = 5)
scores.mean()

/home/robbenm/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


0.02230802415875755

It looks like 9 wins, still a low score but whatevs let's make a model function and then run it.

In [98]:
def make_ada(features,labels,ncores = 1):
    train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                                labels,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
    #Make the random forest
    ada = AdaBoostClassifier(base_estimator=dt9,learning_rate=0.4,n_estimators = 7500)
#     cv_score = cross_val_score(rf, features, labels,cv = 5,n_jobs = 16)
    ada.fit(train_features,train_labels)
    
    
    print('Training score: ', ada.score(train_features, train_labels)*100)
    print('Test score: ', ada.score(test_features, test_labels)*100)
    
    predictions = ada.predict(test_features)
    y_pred = ada.predict_proba(test_features)
    lloss = log_loss(test_labels,y_pred,labels=ada.classes_)
    print('Log Loss: ',lloss)
#     print('Cross validation score: ',cv_score.mean())
    return(ada)

In [99]:
Motif_present_ada = make_ada(Motif_present_features_array,Motif_present_targetEC)

Training score:  93.34500875656742
Test score:  1.574803149606299
Log Loss:  6.649269666903975


In [100]:
Motif_number_ada = make_ada(Motif_number_features_array,Motif_number_targetEC)

Training score:  93.95796847635727
Test score:  1.0498687664041995
Log Loss:  6.883513747390988


In [101]:
Nodes_present_ada = make_ada(Nodes_present_features_array,Nodes_present_targetEC)

Training score:  58.90295358649789
Test score:  8.080808080808081
Log Loss:  4.212237406005534


In [102]:
Nodes_number_ada = make_ada(Nodes_number_features_array,Nodes_number_targetEC)

Training score:  56.87763713080168
Test score:  7.828282828282829
Log Loss:  4.234130539692141


In [103]:
Motif_present_ada = make_ada(Motif_present_features_array,Motif_present_targetMap)

Training score:  89.14185639229422
Test score:  2.6246719160104988
Log Loss:  7.087755014384876


In [104]:
Motif_number_ada = make_ada(Motif_number_features_array,Motif_number_targetMap)

Training score:  90.19264448336253
Test score:  1.0498687664041995
Log Loss:  7.276042964658349


In [105]:
Nodes_present_ada = make_ada(Nodes_present_features_array,Nodes_present_targetMap)

Training score:  23.375527426160335
Test score:  6.8181818181818175
Log Loss:  5.18175028335019


In [106]:
Nodes_number_ada = make_ada(Nodes_number_features_array,Nodes_number_targetMap)

Training score:  23.20675105485232
Test score:  6.313131313131313
Log Loss:  5.1966185575841


In [107]:
def partial_ada(features,labels,ncores = 1):
    train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                                labels,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
    #Make the random forest
    ada = AdaBoostClassifier(base_estimator=dt9,learning_rate=0.4,n_estimators = 7500)
#     cv_score = cross_val_score(rf, features, labels,cv = 5,n_jobs = 16)
    ada.fit(train_features,train_labels)
    predictions = ada.predict(test_features)
    errors = 0
    length = 0
    less = 0
    equal = 0
    more = 0
    for x in range(0,len(predictions)):
        map1 = predictions[x]#['path:map00280', 'path:map00630']
        map2 = test_labels[x]
        map1 = map1.replace('[','').replace(']','').split(',')
        map2 = map2.replace('[','').replace(']','').split(',')
        total = len(set(map1).intersection(map2))
        errors = errors + total
        length = length + len(map2)
        if (total < len(map2)): less = less + 1
        if (total == len(map2)): equal = equal + 1
        if (total < len(map1)): more = more + 1
    print('accuracy was ',errors/length*100,'%')
    print('in ', less/len(test_labels)*100,'% of genes, model did not predict all pathways')
    print('in ', equal/len(test_labels)*100,'% of genes, model predicted all pathways')
    print('in ', more/len(test_labels)*100,'% of genes, model predicted pathways not found in gene')
    

In [108]:
partial_ada(Motif_present_features_array,Motif_present_targetMap)

accuracy was  30.093312597200622 %
in  96.8503937007874 % of genes, model did not predict all pathways
in  3.149606299212598 % of genes, model predicted all pathways
in  96.06299212598425 % of genes, model predicted pathways not found in gene


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.4, n_estimators=7500)

In [109]:
partial_ada(Motif_number_features_array,Motif_number_targetMap)

accuracy was  29.307568438003223 %
in  98.9501312335958 % of genes, model did not predict all pathways
in  1.0498687664041995 % of genes, model predicted all pathways
in  97.9002624671916 % of genes, model predicted pathways not found in gene


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.4, n_estimators=7500)

In [110]:
partial_ada(Nodes_present_features_array,Nodes_present_targetMap)

accuracy was  7.742878013148284 %
in  93.18181818181817 % of genes, model did not predict all pathways
in  6.8181818181818175 % of genes, model predicted all pathways
in  93.18181818181817 % of genes, model predicted pathways not found in gene


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.4, n_estimators=7500)

In [111]:
partial_ada(Nodes_number_features_array,Nodes_number_targetMap)

accuracy was  7.377647918188458 %
in  93.68686868686868 % of genes, model did not predict all pathways
in  6.313131313131313 % of genes, model predicted all pathways
in  93.68686868686868 % of genes, model predicted pathways not found in gene


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.4, n_estimators=7500)

Let's do the Gradient Boost now

In [82]:
clf = HistGradientBoostingClassifier()
pprint(clf.get_params())

{'categorical_features': None,
 'early_stopping': 'auto',
 'l2_regularization': 0.0,
 'learning_rate': 0.1,
 'loss': 'auto',
 'max_bins': 255,
 'max_depth': None,
 'max_iter': 100,
 'max_leaf_nodes': 31,
 'min_samples_leaf': 20,
 'monotonic_cst': None,
 'n_iter_no_change': 10,
 'random_state': None,
 'scoring': 'loss',
 'tol': 1e-07,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}


In [83]:
random_grid = {'learning_rate': [0.1,0.4,0.8,1.2],
              'max_depth': ['None',1,5,10],
              'max_iter': [100,1000,5000,10000],
              'max_leaf_nodes': [20,40,60],
              'min_samples_leaf': [2,10,20,30]}
clf = HistGradientBoostingClassifier()
Motif_present_hist = RandomizedSearchCV(estimator=clf, 
                               param_distributions=random_grid,
                               n_iter=100, 
                               cv = 3, 
                               verbose =2, 
                               random_state=42,
                               n_jobs = 32)
# Motif_present_hist.fit(train_features, train_labels)
# pprint(Motif_present_hist.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/home/robbenm/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


[CV] END learning_rate=0.4, max_depth=None, max_iter=1000, max_leaf_nodes=20, min_samples_leaf=2; total time=   0.1s
[CV] END learning_rate=0.4, max_depth=None, max_iter=1000, max_leaf_nodes=60, min_samples_leaf=2; total time=   0.0s
[CV] END learning_rate=0.4, max_depth=None, max_iter=1000, max_leaf_nodes=60, min_samples_leaf=2; total time=   0.0s
[CV] END learning_rate=0.4, max_depth=None, max_iter=1000, max_leaf_nodes=60, min_samples_leaf=2; total time=   0.0s
[CV] END learning_rate=0.4, max_depth=None, max_iter=100, max_leaf_nodes=40, min_samples_leaf=30; total time=   0.0s
[CV] END learning_rate=0.4, max_depth=None, max_iter=100, max_leaf_nodes=40, min_samples_leaf=30; total time=   0.0s
[CV] END learning_rate=0.4, max_depth=None, max_iter=100, max_leaf_nodes=40, min_samples_leaf=30; total time=   0.0s
[CV] END learning_rate=0.4, max_depth=1, max_iter=5000, max_leaf_nodes=20, min_samples_leaf=10; total time= 7.3min


/home/robbenm/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END learning_rate=1.2, max_depth=1, max_iter=100, max_leaf_nodes=20, min_samples_leaf=2; total time=   9.3s
[CV] END learning_rate=0.4, max_depth=1, max_iter=5000, max_leaf_nodes=20, min_samples_leaf=10; total time= 7.4min
[CV] END learning_rate=0.1, max_depth=5, max_iter=100, max_leaf_nodes=20, min_samples_leaf=10; total time= 1.0min
[CV] END learning_rate=0.8, max_depth=1, max_iter=5000, max_leaf_nodes=20, min_samples_leaf=10; total time= 7.4min
[CV] END learning_rate=0.1, max_depth=5, max_iter=100, max_leaf_nodes=20, min_samples_leaf=10; total time= 1.0min
[CV] END learning_rate=0.8, max_depth=1, max_iter=5000, max_leaf_nodes=20, min_samples_leaf=10; total time= 7.4min
[CV] END learning_rate=1.2, max_depth=5, max_iter=1000, max_leaf_nodes=40, min_samples_leaf=20; total time= 1.5min
[CV] END learning_rate=0.8, max_depth=1, max_iter=5000, max_leaf_nodes=20, min_samples_leaf=10; total time= 7.3min
[CV] END learning_rate=0.4, max_depth=None, max_iter=1000, max_leaf_nodes=20, min_sa

...

[CV] END learning_rate=0.1, max_depth=None, max_iter=100, max_leaf_nodes=60, min_samples_leaf=20; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=None, max_iter=100, max_leaf_nodes=60, min_samples_leaf=20; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=None, max_iter=100, max_leaf_nodes=60, min_samples_leaf=20; total time=   0.1s
[CV] END learning_rate=0.4, max_depth=None, max_iter=100, max_leaf_nodes=20, min_samples_leaf=2; total time=   0.0s
[CV] END learning_rate=0.4, max_depth=None, max_iter=100, max_leaf_nodes=20, min_samples_leaf=2; total time=   0.1s
[CV] END learning_rate=0.4, max_depth=None, max_iter=100, max_leaf_nodes=20, min_samples_leaf=2; total time=   0.1s
[CV] END learning_rate=0.8, max_depth=1, max_iter=1000, max_leaf_nodes=40, min_samples_leaf=2; total time= 1.5min
[CV] END learning_rate=0.1, max_depth=10, max_iter=5000, max_leaf_nodes=40, min_samples_leaf=20; total time=22.7min
[CV] END learning_rate=0.4, max_depth=5, max_iter=10000, max_leaf_nodes

[CV] END learning_rate=1.2, max_depth=1, max_iter=10000, max_leaf_nodes=40, min_samples_leaf=30; total time=14.2min
[CV] END learning_rate=0.4, max_depth=10, max_iter=5000, max_leaf_nodes=20, min_samples_leaf=30; total time= 7.2min
[CV] END learning_rate=0.8, max_depth=5, max_iter=1000, max_leaf_nodes=20, min_samples_leaf=20; total time= 1.4min
[CV] END learning_rate=1.2, max_depth=5, max_iter=1000, max_leaf_nodes=60, min_samples_leaf=2; total time= 2.4min
[CV] END learning_rate=0.8, max_depth=5, max_iter=10000, max_leaf_nodes=20, min_samples_leaf=30; total time=14.6min
[CV] END learning_rate=0.1, max_depth=1, max_iter=5000, max_leaf_nodes=60, min_samples_leaf=10; total time=20.2min
[CV] END learning_rate=0.8, max_depth=5, max_iter=5000, max_leaf_nodes=40, min_samples_leaf=30; total time= 7.3min
[CV] END learning_rate=0.8, max_depth=5, max_iter=100, max_leaf_nodes=20, min_samples_leaf=30; total time=   9.0s
[CV] END learning_rate=0.8, max_depth=5, max_iter=100, max_leaf_nodes=20, min_s

/home/robbenm/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
87 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
87 fits failed with the following error:
Traceback (most recent call last):
  File "/home/robbenm/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/robbenm/.local/lib/python3.8/site-packages/sklearn/ensemble/_hist_gradient_boosting/gradient_boosting.py", line 510, in fit
    grower = TreeGrower(
  File "/home/robbenm/.local/lib/python3.8/site-packages/sklearn/ensemble/_hist_gradient_boosting/grower.py", line 207, in __init__
  

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

[CV] END learning_rate=0.8, max_depth=1, max_iter=10000, max_leaf_nodes=40, min_samples_leaf=10; total time=14.7min
[CV] END learning_rate=0.1, max_depth=1, max_iter=1000, max_leaf_nodes=40, min_samples_leaf=30; total time= 4.4min
[CV] END learning_rate=0.4, max_depth=10, max_iter=10000, max_leaf_nodes=20, min_samples_leaf=30; total time=15.3min
[CV] END learning_rate=0.8, max_depth=None, max_iter=10000, max_leaf_nodes=60, min_samples_leaf=2; total time=   0.0s
[CV] END learning_rate=0.8, max_depth=None, max_iter=10000, max_leaf_nodes=60, min_samples_leaf=2; total time=   0.0s
[CV] END learning_rate=0.8, max_depth=None, max_iter=10000, max_leaf_nodes=60, min_samples_leaf=2; total time=   0.0s
[CV] END learning_rate=0.1, max_depth=10, max_iter=10000, max_leaf_nodes=20, min_samples_leaf=10; total time=41.3min
[CV] END learning_rate=0.8, max_depth=10, max_iter=10000, max_leaf_nodes=40, min_samples_leaf=2; total time=16.5min
[CV] END learning_rate=0.1, max_depth=1, max_iter=5000, max_leaf_

In [84]:
pprint(Motif_present_hist.best_params_)

{'learning_rate': 0.1,
 'max_depth': 1,
 'max_iter': 100,
 'max_leaf_nodes': 40,
 'min_samples_leaf': 20}


Looks like the best options for the gradient is a learning rate of 0.1, max depth 1, max iter 100, max_leaf_nodes 40, and min samples leaf 20. Now lets make it a function and run the calc for every model and predictor.

In [87]:
def make_hist(features,labels,ncores = 1):
    train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                                labels,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
    #Make the random forest
    hist = HistGradientBoostingClassifier(learning_rate = 0.1, max_depth = 1, max_iter = 100, max_leaf_nodes = 40, min_samples_leaf = 20)
#     cv_score = cross_val_score(rf, features, labels,cv = 5,n_jobs = 16)
    hist.fit(train_features,train_labels)
    
    
    print('Training score: ', hist.score(train_features, train_labels)*100)
    print('Test score: ', hist.score(test_features, test_labels)*100)
    
    predictions = hist.predict(test_features)
    y_pred = hist.predict_proba(test_features)
    lloss = log_loss(test_labels,y_pred,labels=hist.classes_)
    print('Log Loss: ',lloss)
#     print('Cross validation score: ',cv_score.mean())
    predictions = hist.predict(test_features)
    
    errors = 0
    length = 0
    less = 0
    equal = 0
    more = 0
    if (ncores > 1):
        for x in range(0,len(predictions)):
            map1 = predictions[x]#['path:map00280', 'path:map00630']
            map2 = test_labels[x]
            map1 = map1.replace('[','').replace(']','').split(',')
            map2 = map2.replace('[','').replace(']','').split(',')
            total = len(set(map1).intersection(map2))
            errors = errors + total
            length = length + len(map2)
            if (total < len(map2)): less = less + 1
            if (total == len(map2)): equal = equal + 1
            if (total < len(map1)): more = more + 1
        print('Partial accuracy was ',errors/length*100,'%')
    return(hist)

In [88]:
Motif_present_hist = make_hist(Motif_present_features_array,Motif_present_targetEC)

Training score:  92.81961471103327
Test score:  4.986876640419948
Log Loss:  6.249081312107145


In [89]:
Motif_number_hist = make_hist(Motif_number_features_array,Motif_number_targetEC)

Training score:  93.52014010507881
Test score:  3.149606299212598
Log Loss:  6.817978785476383


In [90]:
Nodes_present_hist = make_hist(Nodes_present_features_array,Nodes_present_targetEC)

Training score:  0.7594936708860759
Test score:  0.0
Log Loss:  22.904112608943016


In [91]:
Nodes_number_hist = make_hist(Nodes_number_features_array,Nodes_number_targetEC)

Training score:  1.0970464135021099
Test score:  0.25252525252525254
Log Loss:  22.938631797630553


In [92]:
Motif_present_histmap = make_hist(Motif_present_features_array,Motif_present_targetMap,ncores = 3)

Training score:  67.60070052539405
Test score:  6.561679790026247
Log Loss:  5.88255239019566
Partial accuracy was  34.914463452566096 %


In [93]:
Motif_number_histmap = make_hist(Motif_number_features_array,Motif_number_targetMap,ncores = 3)

Training score:  69.43957968476357
Test score:  3.937007874015748
Log Loss:  6.205620534313747
Partial accuracy was  33.41384863123994 %


In [94]:
Nodes_present_histmap = make_hist(Nodes_present_features_array,Nodes_present_targetMap,ncores = 3)

Training score:  14.852320675105485
Test score:  5.808080808080808
Log Loss:  5.123315969752606
Partial accuracy was  36.742147552958365 %


In [2]:
# Nodes_number_histmap = make_hist(Nodes_number_features_array,Nodes_number_targetMap,ncores = 3)

Now go ahead and write a function combining all of the above into one that spits out all the information into a table that you can export into R.

In [34]:
# train_features, test_features, train_labels, test_labels = train_test_split(features,
#                                                                                 labels,
#                                                                                 test_size = 0.25, 
#                                                                                 random_state = 42)
def partial_accuracy(test_labels,predictions):
    errors = 0
    length = 0
    for x in range(0,len(predictions)):
            map1 = predictions[x]#['path:map00280', 'path:map00630']
            map2 = test_labels[x]
            map1 = map1.replace('[','').replace(']','').split(',')
            map2 = map2.replace('[','').replace(']','').split(',')
            total = len(set(map1).intersection(map2))
            errors = errors + total
            length = length + len(map2)
    return(errors/length*100)        
    


def make_Model(train_features,test_features,train_labels,test_labels,name,predictor,partial = False):
    print('Processing ', name, ' Random Forest ',predictor)
    start_time = time.time()
    ######RandomForest
    rf = RandomForestClassifier(random_state = 42,n_estimators = 5000,min_samples_split = 2,min_samples_leaf = 1,max_features = 'sqrt',max_depth = 10,bootstrap = True,oob_score = True)
    rf.fit(train_features,train_labels)
    acc_rf = rf.score(test_features, test_labels)*100
    predictions = rf.predict(test_features)
    y_pred = rf.predict_proba(test_features)
    lloss_rf = log_loss(test_labels,y_pred,labels=rf.classes_)
    conf_mat = confusion_matrix(test_labels, predictions)
    FP_rf = conf_mat.sum(axis=0) - np.diag(conf_mat)  
    FN_rf = conf_mat.sum(axis=1) - np.diag(conf_mat)
    TP_rf = np.diag(conf_mat)
    TN_rf = conf_mat.sum() - (FP_rf + FN_rf + TP_rf)
    if (partial):
        acc_rf = partial_accuracy(test_labels,predictions)
    rf_time = time.time() - start_time
    print('Took ', rf_time, ' seconds')
    
    start_time = time.time()
    print('Processing ', name, ' Ada Boost')
    #######Adaboost
    ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),learning_rate=0.4,n_estimators = 7500)
    ada.fit(train_features,train_labels)
    acc_ada = ada.score(test_features, test_labels)*100
    predictions = ada.predict(test_features)
    y_pred = ada.predict_proba(test_features)
    lloss_ada = log_loss(test_labels,y_pred,labels=ada.classes_)
    conf_mat = confusion_matrix(test_labels, predictions)
    FP_ada = conf_mat.sum(axis=0) - np.diag(conf_mat)  
    FN_ada = conf_mat.sum(axis=1) - np.diag(conf_mat)
    TP_ada = np.diag(conf_mat)
    TN_ada = conf_mat.sum() - (FP_ada + FN_ada + TP_ada)
    if (partial):
        acc_ada = partial_accuracy(test_labels,predictions)
    ada_time = time.time() - start_time
    print('Took ', ada_time, ' seconds')
    
    start_time = time.time()
    print('Processing ', name, ' Gradient Boost')
    ############Gradient Boost
    hist = HistGradientBoostingClassifier(learning_rate = 0.1, max_depth = 1, max_iter = 100, max_leaf_nodes = 40, min_samples_leaf = 20)
    hist.fit(train_features,train_labels)
    acc_hist = hist.score(test_features, test_labels)*100
    predictions = hist.predict(test_features)
    y_pred = hist.predict_proba(test_features)
    lloss_hist = log_loss(test_labels,y_pred,labels=hist.classes_)
    conf_mat = confusion_matrix(test_labels, predictions)
    FP_hist = conf_mat.sum(axis=0) - np.diag(conf_mat)  
    FN_hist = conf_mat.sum(axis=1) - np.diag(conf_mat)
    TP_hist = np.diag(conf_mat)
    TN_hist = conf_mat.sum() - (FP_hist + FN_hist + TP_hist)
    if (partial):
        acc_rf = partial_accuracy(test_labels,predictions)
    
    hist_time = time.time() - start_time
    print('Took ', rf_time, ' seconds')
    
    
    
    df = pd.DataFrame({'Model': ['Random Forest','Ada Boosting','Gradient Boosting'],
                       'Result': [acc_rf,acc_ada,acc_hist],
                       'Loss': [lloss_rf,lloss_ada,lloss_hist],
                       'FP': [FP_rf.mean(),FP_ada.mean(),FP_hist.mean()],
                       'FN': [FN_rf.mean(),FN_ada.mean(),FN_hist.mean()],
                       'TP': [TP_rf.mean(),TP_ada.mean(),TP_hist.mean()],
                       'TN': [TN_rf.mean(),TN_ada.mean(),TN_hist.mean()],
                       'Time': [rf_time,ada_time,hist_time],
                       'Method': [name,name,name],
                       'Predictor': [predictor,predictor,predictor]})
    print('Finished ',name)
    return(df)
    

In [29]:
#Do all for Motif Number first
train_features, test_features, train_labels, test_labels = train_test_split(Motif_number_features_array,
                                                                                Motif_number_targetEC,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
Motif_Number_ecresult = make_Model(train_features,test_features,train_labels,test_labels,'Motif_number','ec_id')
train_features, test_features, train_labels, test_labels = train_test_split(Motif_number_features_array,
                                                                                Motif_number_targetMap,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
Motif_Number_mapresult = make_Model(train_features,test_features,train_labels,test_labels,'Motif_number','map_id')
Motif_Number_partialresult = make_Model(train_features,test_features,train_labels,test_labels,'Motif_number','partial',partial = True)


Processing  Motif_number  Random Forest  ec_id
Took  21.47507929801941  seconds
Processing  Motif_number  Ada Boost
Took  360.5186803340912  seconds
Processing  Motif_number  Gradient Boost
Took  21.47507929801941  seconds
Finished  Motif_number
Processing  Motif_number  Random Forest  map_id
Took  14.717329740524292  seconds
Processing  Motif_number  Ada Boost
Took  170.43010902404785  seconds
Processing  Motif_number  Gradient Boost
Took  14.717329740524292  seconds
Finished  Motif_number
Processing  Motif_number  Random Forest  partial
Took  14.64791488647461  seconds
Processing  Motif_number  Ada Boost
Took  170.6636209487915  seconds
Processing  Motif_number  Gradient Boost
Took  14.64791488647461  seconds
Finished  Motif_number


In [30]:
#Do all for Motif Number first
train_features, test_features, train_labels, test_labels = train_test_split(Motif_present_features_array,
                                                                                Motif_present_targetEC,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
Motif_Present_ecresult = make_Model(train_features,test_features,train_labels,test_labels,'Motif_present','ec_id')
train_features, test_features, train_labels, test_labels = train_test_split(Motif_present_features_array,
                                                                                Motif_present_targetMap,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
Motif_Present_mapresult = make_Model(train_features,test_features,train_labels,test_labels,'Motif_present','map_id')
Motif_Present_partialresult = make_Model(train_features,test_features,train_labels,test_labels,'Motif_present','partial',,partial = True)


SyntaxError: invalid syntax (1208590011.py, line 12)

In [ ]:
#Do all for Motif Number first
train_features, test_features, train_labels, test_labels = train_test_split(Nodes_present_features_array,
                                                                                Nodes_present_targetEC,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
Nodes_Present_ecresult = make_Model(train_features,test_features,train_labels,test_labels,'Nodes_present','ec_id')
train_features, test_features, train_labels, test_labels = train_test_split(Nodes_present_features_array,
                                                                                Nodes_present_targetMap,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
Nodes_Present_mapresult = make_Model(train_features,test_features,train_labels,test_labels,'Nodes_present','map_id')
Nodes_Present_partialresult = make_Model(train_features,test_features,train_labels,test_labels,'Nodes_present','partial',partial = True)


In [ ]:
#Do all for Motif Number first
train_features, test_features, train_labels, test_labels = train_test_split(Nodes_number_features_array,
                                                                                Nodes_number_targetEC,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
Nodes_Number_ecresult = make_Model(train_features,test_features,train_labels,test_labels,'Nodes_number','ec_id')
train_features, test_features, train_labels, test_labels = train_test_split(Nodes_number_features_array,
                                                                                Nodes_number_targetMap,
                                                                                test_size = 0.25, 
                                                                                random_state = 42)
Nodes_Number_mapresult = make_Model(train_features,test_features,train_labels,test_labels,'Nodes_number','map_id')
Nodes_Number_partialresult = make_Model(train_features,test_features,train_labels,test_labels,'Nodes_number','partial',partial = True)


In [ ]:
Train_results_df = pd.concat([Motif_Number_ecresult,Motif_Number_mapresult,Motif_Number_partialresult,
                 Motif_Present_ecresult,Motif_Present_mapresult,Motif_Present_partialresult,
                 Nodes_Number_ecresult,Nodes_Number_mapresult,Nodes_Number_partialresult,
                 Nodes_Present_ecresult,Nodes_Present_mapresult,Nodes_Present_partialresult,])
print(Train_results_df)
Train_results_df.to_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Training_results_Ensemble.csv')

Now we can move on the the actual test set in which we will try to classify a set of annotated genes from smn.

In [31]:
#Read in the new datasets
Motif_present_smn = pd.read_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/SMN_Test/Motif_Present_smn.tsv',sep="\t")
Motif_number_smn = pd.read_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/SMN_Test/Motif_Number_smn.tsv',sep="\t")
Nodes_present_smn = pd.read_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/SMN_Test/Nodes_Present_smn.tsv',sep="\t")
Nodes_number_smn = pd.read_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/SMN_Test/Nodes_Number_smn.tsv',sep="\t")
Motif_number_smn.head(5)

,gene_id,protein_id,ec_id,species,map_id,MEME-1,MEME-2,MEME-3,MEME-4,MEME-5,...,MEME-88,MEME-89,MEME-90,MEME-91,MEME-92,MEME-93,MEME-94,MEME-95,MEME-96,MEME-97
0,smn:SMA_0012,ncbi-proteinid:CCF01303,ec:3.5.2.6,smn,"['path:map00311', 'path:map01110']",0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,smn:SMA_0014,ncbi-proteinid:CCF01305,ec:2.4.2.8,smn,"['path:map00230', 'path:map00983', 'path:map01...",0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,smn:SMA_0023,ncbi-proteinid:CCF01314,ec:2.7.6.1,smn,"['path:map00030', 'path:map00230', 'path:map01...",1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,smn:SMA_0026,ncbi-proteinid:CCF01317,ec:2.3.1.274,smn,"['path:map00561', 'path:map01100']",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,smn:SMA_0028,ncbi-proteinid:CCF01319,ec:6.3.2.6,smn,"['path:map00230', 'path:map01100', 'path:map01...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
Motif_present_targetEC_smn = np.array(Motif_present_smn['ec_id']) #Lets start with predicting the EC and then we can try mapID later
Motif_number_targetEC_smn = np.array(Motif_number_smn['ec_id'])
Nodes_present_targetEC_smn = np.array(Nodes_present_smn['ec_id'])
Nodes_number_targetEC_smn = np.array(Nodes_number_smn['ec_id'])
Motif_number_targetEC_smn.size

356

In [33]:
Motif_present_targetMap_smn = np.array(Motif_present_smn['map_id'])
Motif_number_targetMap_smn = np.array(Motif_number_smn['map_id'])
Nodes_present_targetMap_smn = np.array(Nodes_present_smn['map_id'])
Nodes_number_targetMap_smn = np.array(Nodes_number_smn['map_id'])
Motif_present_targetMap_smn.size

356

In [34]:
#Drop columns from feature array
colDrop = ['gene_id','protein_id','ec_id','species','map_id']
Motif_present_features_smn = Motif_present_smn.drop(colDrop,axis = 1)
Motif_number_features_smn = Motif_number_smn.drop(colDrop,axis = 1)
Nodes_present_features_smn = Nodes_present_smn.drop(colDrop,axis = 1)
Nodes_number_features_smn = Nodes_number_smn.drop(colDrop,axis = 1)
#convert to array
Motif_present_features_array_smn = np.array(Motif_present_features_smn)
Motif_number_features_array_smn = np.array(Motif_number_features_smn)
Nodes_present_features_array_smn = np.array(Nodes_present_features_smn)
Nodes_number_features_array_smn = np.array(Nodes_number_features_smn)

Now that we have these in we can run the models using all the ecobis set as training data and all the smn set as a test set. 

In [35]:
#Do all for Motif Number first
Motif_Number_ecresult = make_Model(Motif_number_features_array,Motif_number_features_array_smn,Motif_number_targetEC,Motif_number_targetEC_smn,'Motif_number','ec_id')
Motif_Number_mapresult = make_Model(Motif_number_features_array,Motif_number_features_array_smn,Motif_number_targetMap,Motif_number_targetMap_smn,'Motif_number','map_id')
Motif_Number_partialresult = make_Model(Motif_number_features_array,Motif_number_features_array_smn,Motif_number_targetMap,Motif_number_targetMap_smn,'Motif_number','partial',partial = True)
#Do all for Motif Present
Motif_Present_ecresult = make_Model(Motif_present_features_array,Motif_present_features_array_smn,Motif_present_targetEC,Motif_present_targetEC_smn,'Motif_present','ec_id')
Motif_Present_mapresult = make_Model(Motif_present_features_array,Motif_present_features_array_smn,Motif_present_targetMap,Motif_present_targetMap_smn,'Motif_present','map_id')
Motif_Present_partialresult = make_Model(Motif_present_features_array,Motif_present_features_array_smn,Motif_present_targetMap,Motif_present_targetMap_smn,'Motif_present','partial',partial = True)
#Do all for Nodes Present
Nodes_Present_ecresult = make_Model(Nodes_present_features_array,Nodes_present_features_array_smn,Nodes_present_targetEC,Nodes_present_targetEC_smn,'Nodes_present','ec_id')
Nodes_Present_mapresult = make_Model(Nodes_present_features_array,Nodes_present_features_array_smn,Nodes_present_targetMap,Nodes_present_targetMap_smn,'Nodes_present','map_id')
Nodes_Present_partialresult = make_Model(Nodes_present_features_array,Nodes_present_features_array_smn,Nodes_present_targetMap,Nodes_present_targetMap_smn,'Nodes_present','partial',partial = True)
#Do all for Nodes Number
Nodes_Number_ecresult = make_Model(Nodes_number_features_array,Nodes_number_features_array_smn,Nodes_number_targetEC,Nodes_number_targetEC_smn,'Nodes_number','ec_id')
Nodes_Number_mapresult = make_Model(Nodes_number_features_array,Nodes_number_features_array_smn,Nodes_number_targetMap,Nodes_number_targetMap_smn,'Nodes_number','map_id')
Nodes_Number_partialresult = make_Model(Nodes_number_features_array,Nodes_number_features_array_smn,Nodes_number_targetMap,Nodes_number_targetMap_smn,'Nodes_number','partial',partial = True)


Processing  Motif_number  Random Forest  ec_id
Took  27.505934476852417  seconds
Processing  Motif_number  Ada Boost
Took  522.4465916156769  seconds
Processing  Motif_number  Gradient Boost
Took  27.505934476852417  seconds
Finished  Motif_number
Processing  Motif_number  Random Forest  map_id
Took  16.041162729263306  seconds
Processing  Motif_number  Ada Boost
Took  221.15613627433777  seconds
Processing  Motif_number  Gradient Boost
Took  16.041162729263306  seconds
Finished  Motif_number
Processing  Motif_number  Random Forest  partial
Took  16.250980615615845  seconds
Processing  Motif_number  Ada Boost
Took  222.3646776676178  seconds
Processing  Motif_number  Gradient Boost
Took  16.250980615615845  seconds
Finished  Motif_number
Processing  Motif_present  Random Forest  ec_id
Took  27.224644660949707  seconds
Processing  Motif_present  Ada Boost
Took  495.9595637321472  seconds
Processing  Motif_present  Gradient Boost
Took  27.224644660949707  seconds
Finished  Motif_present


In [36]:
Test_results_df = pd.concat([Motif_Number_ecresult,Motif_Number_mapresult,Motif_Number_partialresult,
                 Motif_Present_ecresult,Motif_Present_mapresult,Motif_Present_partialresult,
                 Nodes_Number_ecresult,Nodes_Number_mapresult,Nodes_Number_partialresult,
                 Nodes_Present_ecresult,Nodes_Present_mapresult,Nodes_Present_partialresult,])
print(Test_results_df)
Test_results_df.to_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Testing_results_Ensemble.csv')

               Model     Result       Loss        FP        FN        TP  \
0      Random Forest   0.000000   5.539220  1.005650  1.005650  0.000000   
1       Ada Boosting   0.000000   8.393204  0.874693  0.874693  0.000000   
2  Gradient Boosting   0.561798   8.126514  0.793722  0.793722  0.004484   
0      Random Forest   0.561798   4.785904  1.945055  1.945055  0.010989   
1       Ada Boosting   0.842697   6.756687  1.773869  1.773869  0.015075   
2  Gradient Boosting   0.561798   5.584271  1.539130  1.539130  0.008696   
0      Random Forest  28.778822   4.785904  1.945055  1.945055  0.010989   
1       Ada Boosting  28.522630   6.754840  1.786802  1.786802  0.020305   
2  Gradient Boosting   0.561798   5.584271  1.539130  1.539130  0.008696   
0      Random Forest   0.000000   5.542554  1.017143  1.017143  0.000000   
1       Ada Boosting   0.000000   8.396381  0.874693  0.874693  0.000000   
2  Gradient Boosting   0.561798   8.126514  0.791946  0.791946  0.004474   
0      Rando

In [3]:
# rf = RandomForestClassifier(random_state = 42,
#                                 n_estimators = 5000,
#                                 min_samples_split = 2,
#                                 min_samples_leaf = 1,
#                                 max_features = 'sqrt',
#                                 max_depth = 10,
#                                 bootstrap = True)
# rf.fit(Motif_number_features_array,Motif_number_targetEC)
# predictions = rf.predict(Motif_number_features_array_smn)
#Motif_number_features_array,Motif_number_features_array_smn,Motif_number_targetEC,Motif_number_targetEC_smn,'Motif_number','ec_id')

In [35]:
# train_features, test_features, train_labels, test_labels = train_test_split(features,
#                                                                                 labels,
#                                                                                 test_size = 0.25, 
#                                                                                 random_state = 42)
def partial_accuracy(test_labels,predictions):
    errors = 0
    length = 0
    for x in range(0,len(predictions)):
            map1 = predictions[x]#['path:map00280', 'path:map00630']
            map2 = test_labels[x]
            map1 = map1.replace('[','').replace(']','').split(',')
            map2 = map2.replace('[','').replace(']','').split(',')
            total = len(set(map1).intersection(map2))
            errors = errors + total
            length = length + len(map2)
    return(errors/length*100)        
    


def make_Model(train_features,test_features,train_labels,test_labels,name,predictor,partial = False):
    print('Processing ', name, ' Random Forest ',predictor)
    start_time = time.time()
    ######RandomForest
    rf = RandomForestClassifier(random_state = 42,n_estimators = 5000,min_samples_split = 2,min_samples_leaf = 1,max_features = 'sqrt',max_depth = 10,bootstrap = True,oob_score = True)
    rf.fit(train_features,train_labels)
    acc_rf = rf.score(test_features, test_labels)*100
    predictions_rf = rf.predict(test_features)
    y_pred = rf.predict_proba(test_features)
    lloss_rf = log_loss(test_labels,y_pred,labels=rf.classes_)
    conf_mat = confusion_matrix(test_labels, predictions_rf)
    FP_rf = conf_mat.sum(axis=0) - np.diag(conf_mat)  
    FN_rf = conf_mat.sum(axis=1) - np.diag(conf_mat)
    TP_rf = np.diag(conf_mat)
    TN_rf = conf_mat.sum() - (FP_rf + FN_rf + TP_rf)
    if (partial):
        acc_rf = partial_accuracy(test_labels,predictions_rf)
    rf_time = time.time() - start_time
    print('Took ', rf_time, ' seconds')
    
    start_time = time.time()
    print('Processing ', name, ' Ada Boost')
    #######Adaboost
    ada = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),learning_rate=0.4,n_estimators = 7500)
    ada.fit(train_features,train_labels)
    acc_ada = ada.score(test_features, test_labels)*100
    predictions_ada = ada.predict(test_features)
    y_pred = ada.predict_proba(test_features)
    lloss_ada = log_loss(test_labels,y_pred,labels=ada.classes_)
    conf_mat = confusion_matrix(test_labels, predictions_ada)
    FP_ada = conf_mat.sum(axis=0) - np.diag(conf_mat)  
    FN_ada = conf_mat.sum(axis=1) - np.diag(conf_mat)
    TP_ada = np.diag(conf_mat)
    TN_ada = conf_mat.sum() - (FP_ada + FN_ada + TP_ada)
    if (partial):
        acc_ada = partial_accuracy(test_labels,predictions_ada)
    ada_time = time.time() - start_time
    print('Took ', ada_time, ' seconds')
    
    start_time = time.time()
    print('Processing ', name, ' Gradient Boost')
    ############Gradient Boost
    hist = HistGradientBoostingClassifier(learning_rate = 0.1, max_depth = 1, max_iter = 100, max_leaf_nodes = 40, min_samples_leaf = 20)
    hist.fit(train_features,train_labels)
    acc_hist = hist.score(test_features, test_labels)*100
    predictions_hist = hist.predict(test_features)
    y_pred = hist.predict_proba(test_features)
    lloss_hist = log_loss(test_labels,y_pred,labels=hist.classes_)
    conf_mat = confusion_matrix(test_labels, predictions_hist)
    FP_hist = conf_mat.sum(axis=0) - np.diag(conf_mat)  
    FN_hist = conf_mat.sum(axis=1) - np.diag(conf_mat)
    TP_hist = np.diag(conf_mat)
    TN_hist = conf_mat.sum() - (FP_hist + FN_hist + TP_hist)
    if (partial):
        acc_rf = partial_accuracy(test_labels,predictions_hist)
    
    hist_time = time.time() - start_time
    print('Took ', rf_time, ' seconds')
    
    
    df = pd.DataFrame({'RFpred': [predictions_rf],
                       'Adapred': [predictions_ada],
                       'Histpred': [predictions_hist]})
    print('Finished ',name)
    return(df)

In [36]:
#Do all for Motif Number first
Motif_Number_ecresult = make_Model(Motif_number_features_array,Motif_number_features_array_smn,Motif_number_targetEC,Motif_number_targetEC_smn,'Motif_number','ec_id')
#Motif_Number_mapresult = make_Model(Motif_number_features_array,Motif_number_features_array_smn,Motif_number_targetMap,Motif_number_targetMap_smn,'Motif_number','map_id')
#Motif_Number_partialresult = make_Model(Motif_number_features_array,Motif_number_features_array_smn,Motif_number_targetMap,Motif_number_targetMap_smn,'Motif_number','partial',partial = True)
#Do all for Motif Present
Motif_Present_ecresult = make_Model(Motif_present_features_array,Motif_present_features_array_smn,Motif_present_targetEC,Motif_present_targetEC_smn,'Motif_present','ec_id')
#Motif_Present_mapresult = make_Model(Motif_present_features_array,Motif_present_features_array_smn,Motif_present_targetMap,Motif_present_targetMap_smn,'Motif_present','map_id')
#Motif_Present_partialresult = make_Model(Motif_present_features_array,Motif_present_features_array_smn,Motif_present_targetMap,Motif_present_targetMap_smn,'Motif_present','partial',partial = True)
#Do all for Nodes Present
Nodes_Present_ecresult = make_Model(Nodes_present_features_array,Nodes_present_features_array_smn,Nodes_present_targetEC,Nodes_present_targetEC_smn,'Nodes_present','ec_id')
#Nodes_Present_mapresult = make_Model(Nodes_present_features_array,Nodes_present_features_array_smn,Nodes_present_targetMap,Nodes_present_targetMap_smn,'Nodes_present','map_id')
#Nodes_Present_partialresult = make_Model(Nodes_present_features_array,Nodes_present_features_array_smn,Nodes_present_targetMap,Nodes_present_targetMap_smn,'Nodes_present','partial',partial = True)
#Do all for Nodes Number
Nodes_Number_ecresult = make_Model(Nodes_number_features_array,Nodes_number_features_array_smn,Nodes_number_targetEC,Nodes_number_targetEC_smn,'Nodes_number','ec_id')
#Nodes_Number_mapresult = make_Model(Nodes_number_features_array,Nodes_number_features_array_smn,Nodes_number_targetMap,Nodes_number_targetMap_smn,'Nodes_number','map_id')
#Nodes_Number_partialresult = make_Model(Nodes_number_features_array,Nodes_number_features_array_smn,Nodes_number_targetMap,Nodes_number_targetMap_smn,'Nodes_number','partial',partial = True)


Processing  Motif_number  Random Forest  ec_id
Took  28.683926582336426  seconds
Processing  Motif_number  Ada Boost
Took  515.5483901500702  seconds
Processing  Motif_number  Gradient Boost
Took  28.683926582336426  seconds
Finished  Motif_number
Processing  Motif_present  Random Forest  ec_id
Took  27.68413996696472  seconds
Processing  Motif_present  Ada Boost
Took  503.62461376190186  seconds
Processing  Motif_present  Gradient Boost
Took  27.68413996696472  seconds
Finished  Motif_present
Processing  Nodes_present  Random Forest  ec_id
Took  68.85148596763611  seconds
Processing  Nodes_present  Ada Boost
Took  1682.8307247161865  seconds
Processing  Nodes_present  Gradient Boost
Took  68.85148596763611  seconds
Finished  Nodes_present
Processing  Nodes_number  Random Forest  ec_id
Took  71.86184525489807  seconds
Processing  Nodes_number  Ada Boost
Took  2008.9879524707794  seconds
Processing  Nodes_number  Gradient Boost
Took  71.86184525489807  seconds
Finished  Nodes_number


In [37]:
Test_results_df = pd.concat([Motif_Number_ecresult,#Motif_Number_mapresult,Motif_Number_partialresult,
                 Motif_Present_ecresult,#Motif_Present_mapresult,Motif_Present_partialresult,
                 Nodes_Number_ecresult,#Nodes_Number_mapresult,Nodes_Number_partialresult,
                 Nodes_Present_ecresult])#,Nodes_Present_mapresult,Nodes_Present_partialresult,])
print(Test_results_df)
Test_results_df.to_csv('/home/robbenm/LuberLab/crispri/EcoBisTest/Testing_pred_Ensemble.csv')

                                              RFpred  \
0  [ec:7.1.1.2, ec:7.1.1.2, ec:7.1.1.2, ec:7.1.1....   
0  [ec:7.1.1.2, ec:7.1.1.2, ec:7.1.1.2, ec:7.1.1....   
0  [ec:7.1.1.2, ec:7.1.1.2, ec:5.1.3.3, ec:7.1.1....   
0  [ec:6.2.1.3, ec:6.2.1.3, ec:3.2.1.23, ec:2.8.1...   

                                             Adapred  \
0  [ec:3.5.1.1, ec:3.5.1.1, ec:7.1.1.2, ec:2.5.1....   
0  [ec:3.5.1.1, ec:3.5.1.1, ec:7.1.1.2, ec:2.5.1....   
0  [ec:7.1.1.2, ec:7.1.1.2, ec:3.6.1.13, ec:4.6.1...   
0  [ec:2.8.1.4, ec:2.8.1.4, ec:2.5.1.72, ec:3.2.1...   

                                            Histpred  
0  [ec:1.4.3.5, ec:1.4.3.5, ec:1.17.1.8, ec:2.5.1...  
0  [ec:1.4.3.5, ec:1.4.3.5, ec:1.17.1.8, ec:2.5.1...  
0  [ec:2.7.1.200, ec:2.7.1.200, ec:4.1.3.27, ec:3...  
0  [ec:2.7.7.24, ec:2.7.7.24, ec:4.2.1.46, ec:2.7...  
